In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np 
import os
import time

# download artist dataset
initial = input("Enter the name of the artist (include spaces):")
name = initial.replace(" ", "_")
path_to_file = '{}_untrained.txt'.format(name)

# read, then decode from py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

# look at the first 250 characters in the text
print(text[:250])

# unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))


# PROCESS AND VECOTRIZE TEXT

# Creating a mapping from unique characters to indicies
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])


Enter the name of the artist (include spaces):taylor swift
Length of text: 127159 characters
I don't like your little games
Don't like your tilted stage
The role you made me play
Of the fool, no, I don't like you
I don't like your perfect crime
How you laugh when you lie
You said the gun was mine
Isn't cool, no, I don't like you (oh!)

But I
75 unique characters


In [ ]:
# PREDICTION

# create training examples and targets

# So break the text into chunks of seq_length+1. For example, 
# say seq_length is 4 and our text is "Hello". 
# The input sequence would be "Hell", and the target sequence "ello".

# the maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epock = len(text) #seq_length+1

# create training examples/targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# batch command converts individual characters to sequences of desired size
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)


# method to duplicate and shift it to form the input and target text
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text


dataset = sequences.map(split_input_target)

for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))


for i, (input_idx, target_idx) in enumerate(zip(input_example[:2], target_example[:2])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))  

Input data:  "I don't like your little games\nDon't like your tilted stage\nThe role you made me play\nOf the fool, n"
Target data: " don't like your little games\nDon't like your tilted stage\nThe role you made me play\nOf the fool, no"
Step    0
  input: 28 ('I')
  expected output: 1 (' ')
Step    1
  input: 1 (' ')
  expected output: 50 ('d')


In [ ]:
# CREATE TRAINING BATCHES

# use tf.data to split text into manageable sequences, need to shuffle data and pack into batches beforehand

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# BUILD MODEL

# use tf.keras.Sequential to define model
# tf.keras.layers.Embedding: input layer, map numbers of each character to vector with embedding_dim dimensions
# tf.keras.layers.GRU: RNN with size units=rnn_units
# tf.keras.layers.Dense: Output layer with vocab_size outputs

# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

#check shape
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

model.summary()



#sample batch

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

sampled_indices

print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

(64, 100, 75) # (batch_size, sequence_length, vocab_size)
Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (64, None, 256)           19200     
_________________________________________________________________
gru_16 (GRU)                 (64, None, 1024)          3938304   
_________________________________________________________________
dense_16 (Dense)             (64, None, 75)            76875     
Total params: 4,034,379
Trainable params: 4,034,379
Non-trainable params: 0
_________________________________________________________________
Input: 
 "till have me\nYou can see that I've been cryin'\nBaby, you know all the right things\nTo say\nBut do you"

Next Char Predictions: 
 'N]K"O[7g?E-l0D4UsQTl1ULcDoQó JrLm1Fk.F((YDYCBedR?K,euzz[pYOY6z)GO7déYX7u "v]]2Uumb1Mrv)OzCY7zeSX3Dov'


In [ ]:
# TRAIN MODEL

# attach optimizer, and a loss function
# tf.keras.losses.sparse_categorical_crossentropy loss function
# logits = from_logits flag

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())


# Configure training procedure
model.compile(optimizer='adam', loss=loss, metrics= ['accuracy'])

# Configure checkpoints

# tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True)

Prediction shape:  (64, 100, 75)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.3180876


In [ ]:
# EXECUTE TRAINING

# uncomment to delete folder
#!rm -rf 'training_checkpoints'


# use 10 epocks to train model, 1 epoch = period of time of a person's life
EPOCHS = 30
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
19/19 [==============================] - 1s 45ms/step - loss: 0.6213 - accuracy: 0.8339
Epoch 2/30
19/19 [==============================] - 1s 47ms/step - loss: 0.5732 - accuracy: 0.8515
Epoch 3/30
19/19 [==============================] - 1s 44ms/step - loss: 0.5291 - accuracy: 0.8667
Epoch 4/30
19/19 [==============================] - 1s 46ms/step - loss: 0.4901 - accuracy: 0.8812
Epoch 5/30
19/19 [==============================] - 1s 46ms/step - loss: 0.4562 - accuracy: 0.8936
Epoch 6/30
19/19 [==============================] - 1s 44ms/step - loss: 0.4222 - accuracy: 0.9056
Epoch 7/30
19/19 [==============================] - 1s 44ms/step - loss: 0.3957 - accuracy: 0.9145
Epoch 8/30
19/19 [==============================] - 1s 47ms/step - loss: 0.3675 - accuracy: 0.9226
Epoch 9/30
19/19 [==============================] - 1s 45ms/step - loss: 0.3472 - accuracy: 0.9293
Epoch 10/30
19/19 [==============================] - 1s 46ms/step - loss: 0.3271 - accuracy: 0.9351
Epoch 11/

In [ ]:
# GENERATE TEXT

# rebuild and restore weights from checkpoint to run with different batch_size
tf.train.latest_checkpoint(checkpoint_dir)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
#print(history)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()


Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (1, None, 256)            19200     
_________________________________________________________________
gru_17 (GRU)                 (1, None, 1024)           3938304   
_________________________________________________________________
dense_17 (Dense)             (1, None, 75)             76875     
Total params: 4,034,379
Trainable params: 4,034,379
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# PREDICTION LOOP

def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 2000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

# write output to single text file
# creates outputs based on helpfulwords and combines into one file
helpfulWords= [u"oh ", u"now ", u"never ", u"I ", u"so "]
filename = '{}_trained_output.txt'.format(name)
out_text = open(filename, "w+")
for word in helpfulWords:
    print("starting word: '{}' generating lyrics...-> {}".format(word,filename))
    out_text.write(generate_text(model, start_string = word))
out_text.close()


starting word: 'oh ' generating lyrics...-> taylor_swift_trained_output.txt
starting word: 'now ' generating lyrics...-> taylor_swift_trained_output.txt
starting word: 'never ' generating lyrics...-> taylor_swift_trained_output.txt
starting word: 'I ' generating lyrics...-> taylor_swift_trained_output.txt
starting word: 'so ' generating lyrics...-> taylor_swift_trained_output.txt
